In [27]:
from qdrant_client import QdrantClient, models
client = QdrantClient("http://localhost:6333") #connecting to local Qdrant instance

In [28]:
from fastembed import TextEmbedding
model_handle = 'jinaai/jina-embeddings-v2-small-en'
embedder = TextEmbedding(model_name=model_handle)

In [29]:
import numpy as np

query = "I just discovered the course. Can I join now?"
vec = next(embedder.embed(query))     
print(vec.shape)                    
print("Q1: What's the minimal value in this array? A: " + str(vec.min()))                


(512,)
Q1: What's the minimal value in this array? A: -0.11726373885183883


In [30]:
query2 = "Can I still join the course after the start date?"
vec2 = next(embedder.embed(query2)) 


np.linalg.norm(vec2)
print("Q2: What's the cosine similarity between the vector for the query and the vector for the document?")
print(float(vec.dot(vec2)))

Q2: What's the cosine similarity between the vector for the query and the vector for the document?
0.9008528895674548


In [31]:
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]

In [32]:
texts = [d["text"] for d in documents]  
V = np.vstack(list(embedder.embed(texts)))  
sims = V.dot(vec)
print(sims)
best_idx = int(np.argmax(sims)) 
print("Q3: Highest-similarity document index:", best_idx)

[0.76296847 0.81823782 0.80853974 0.7133079  0.73044992]
Q3: Highest-similarity document index: 1


In [33]:
full_text = [doc['question'] + ' ' + doc['text'] for doc in documents] 
V2 = np.vstack(list(embedder.embed(full_text)))  
sims2 = V2.dot(vec)
print(sims2)
best_idx2 = int(np.argmax(sims2)) 
print("Q4: Highest-similarity document index:", best_idx2)


[0.85145432 0.84365942 0.8408287  0.7755158  0.80860078]
Q4: Highest-similarity document index: 0


In [34]:
embedder = TextEmbedding("BAAI/bge-small-en")   # vectors are length-384

vec = next(embedder.embed(["test sentence"]))
print("Q5: What's the smallest dimensionality for models in fastembed? " + str(vec.shape)) 

Q5: What's the smallest dimensionality for models in fastembed? (384,)


In [35]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()


documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [ ]:
model_handle = "BAAI/bge-small-en"
EMBEDDING_DIMENSIONALITY = 384

# Define the collection name
collection_name = "zoomcamp-rag3"

# Create the collection with specified vector parameters
client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIMENSIONALITY,  # Dimensionality of the vectors
        distance=models.Distance.COSINE  # Distance metric for similarity search
    )
)



UnexpectedResponse: Unexpected Response: 409 (Conflict)
Raw response content:
b'{"status":{"error":"Wrong input: Collection `zoomcamp-rag2` already exists!"},"time":0.001672903}'

In [ ]:
points = []
id = 0

for course in documents_raw:
    for doc in course['documents']:
        text = doc['question'] + ' ' + doc['text']
        point = models.PointStruct(
            id=id,
            vector=models.Document(text=text, model=model_handle),
            payload={
                "text": doc['text'],
                "section": doc['section'],
                "course": course['course']
            } #save all needed metadata fields
        )
        points.append(point)

        id += 1

In [ ]:
client.upsert(
    collection_name=collection_name,
    points=points
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [ ]:
def search(query, limit=1):

    results = client.query_points(
        collection_name=collection_name,
        query=models.Document( 
            text=query,
            model=model_handle 
        ),
        limit=limit, # top closest matches
        with_payload=True #to get metadata in the results
    )

    return results

In [ ]:
query = "I just discovered the course. Can I join now?"
result = search(query)
print("Q6: What's the highest score in the results? " + str(result.points[0].score))

Q6: What's the highest score in the results? 0.8703172
